# Linear Discrete-Time Quantum Walks (DTQWs)

Linear discrete-time quantum walks perform a quantum walk on a line graph, where the vertices represent states and the edges represent possible transitions between states. The walk is influenced by quantum operators, including diffusion and swap operations, which help in exploring the graph's structure.

The positions the walker can occupy are represented by integers on a line, and the walker can move left or right based on the outcome of a quantum coin flip. This means the positions the walker can occupy are represented by integers on a line, and the walker can move left or right based on the outcome of a quantum coin flip.

![](quantum_walk.jpg)

Components of Linear DTQWs

### 1. Position Space
- The position space consists of states represents the integer positions on the line.
- The walker’s position is described by a superposition of these position states.

### 2. Coin Space
- The coin space is typically a two-dimensional Hilbert space with basis states \(|0\rangle\) (move left) and \(|1\rangle\) (move right).
- The state of the coin determines the direction of the walker’s movement.

### 3. Initial State
- The initial state of the walker is often set to be localized at a particular position, such as the origin, and in a superposition of the coin states.
- Example: The walker starts at position 0 with equal probability to move left or right.

## Operations in Linear DTQWs

### 1. Coin Flip Operation
- A unitary coin operator, C, is applied to the coin space. The most common coin operator is the Hadamard gate:

- This operation creates a superposition of the coin states.

### 2. Shift Operation
- The shift operator, S, moves the walker left or right based on the coin state.

### First Step
- Apply the coin operator (Hadamard gate):

- Apply the shift operator:

### Subsequent Steps
- Repeat the process for the desired number of steps, applying the coin and shift operators iteratively.

## Key Properties and Advantages

- **Superposition and Interference**: Linear DTQWs leverage quantum superposition and interference, resulting in probability distributions that spread faster than classical random walks.
  
- **Spread and Speed**: The walker’s probability distribution spreads quadratically faster than in a classical random walk, leading to potential speedups in algorithms.
  
- **Applications**: Linear DTQWs are useful in quantum search algorithms, exploring structured data, and simulating physical systems.

## Summary

Linear DTQWs involve a quantum walker moving on a one-dimensional line, with movement decisions made based on a quantum coin flip. The walk evolves through iterative application of coin and shift operators, creating complex probability distributions due to quantum superposition and interference. This unique behavior enables faster exploration of state spaces and has various applications in quantum computing and algorithm design.



Imports

In [20]:
from classiq import *
 
size = 4

Function to prepare a qubit in the |-> state

In [21]:
@qfunc
def prepare_minus(x: QBit):
    # Apply the X gate to flip the qubit
    X(x)
    # Apply the H (Hadamard) gate to put the qubit in superposition
    H(x)

Diffuser oracle function

In [22]:
@qfunc
def diffuzer_oracle(aux: Output[QNum], x: QNum):
    # If x is not zero, flip the auxiliary qubit
    aux ^= (x != 0)

Zero diffuser function

In [23]:
@qfunc
def zero_diffuzer(x: QNum):
    aux = QNum('aux')
    # Allocate a single qubit for the auxiliary register
    allocate(1, aux)
    # Apply the diffuser oracle within the context of the |-> state preparation
    within_apply(
        compute=lambda: prepare_minus(aux),
        action=lambda: diffuzer_oracle
    )


In [24]:
# Function to perform a W iteration for a given state i
def W_iteration(i: int, vertices: QNum, adjacent_vertices: QNum):
    # Initialize a probability vector for the state transition
    prob = [0] * 16
    if i == 0:
        prob[i + 1] = 1.0
    elif i == 15:
        prob[i - 1] = 1.0
    else:
        prob[i - 1] = 0.5
        prob[i + 1] = 0.5
    print(f'State={i}, prob vec ={prob}')

        # Apply controlled operations based on the current state
    control(
        ctrl=vertices == i,
        operand=lambda: within_apply(
            compute=lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
            action=lambda: zero_diffuzer(adjacent_vertices)
        )
    )


Function to apply the W operator on the vertices and adjacent_vertices

In [25]:
@qfunc
def W_operator(vertices: QNum, adjacent_vertices: QNum):
    # Iterate over all possible states
    for i in range(2 ** size):
        W_iteration(i, vertices, adjacent_vertices)

Edge oracle function to determine if two vertices are adjacent

In [26]:
@qfunc
def edge_oracle(res: Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    res |= ((vertices - adjacent_vertices) == 1) | ((adjacent_vertices - vertices) == 1)


Function to swap two quantum arrays bitwise

In [27]:
@qfunc
def bitwise_swap(x: QArray[QBit], y: QArray[QBit]):
    repeat(count=x.len, iteration=lambda i: SWAP(x[i], y[i]))

S operator function to apply swaps based on the edge oracle result

In [28]:
@qfunc
def S_operator(vertices: QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res, vertices, adjacent_vertices)
    control(
        ctrl=res == 1,
        operand=lambda: bitwise_swap(vertices, adjacent_vertices)
    )

Main function to initialize and execute the quantum operations

In [29]:
@qfunc
def main(vertices: Output[QNum], adjacent_vertices: Output[QNum]):
    # Allocate qubits for the vertices
    allocate(size, vertices)
    # Apply Hadamard transform to put the vertices in superposition
    hadamard_transform(vertices)
    # Allocate qubits for the adjacent vertices
    allocate(size, adjacent_vertices)
    
    # Apply the W and S operators
    W_operator(vertices, adjacent_vertices)
    S_operator(vertices, adjacent_vertices)

Create and synthesize the quantum model

In [30]:
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

State=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0